### Original Kernel with minimal variations

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.set_printoptions(threshold=200)
pd.set_option('display.max_columns', 200)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data/"))

# Any results you write to the current directory are saved as output.

['device_data.csv', 'pageviews_complemento.csv', 'sampleSubmission.csv', 'SITE_ID.csv', 'PAGE.csv', 'README.md', 'CONTENT_CATEGORY.csv', 'CONTENT_CATEGORY_BOTTOM.csv', 'pageviews.csv', 'conversiones.csv', 'CONTENT_CATEGORY_TOP.csv']


In [81]:
data = pd.concat([
       pd.read_csv("../data/pageviews.csv", parse_dates=["FEC_EVENT"]),
       pd.read_csv("../data/pageviews_complemento.csv", parse_dates=["FEC_EVENT"])
])

/home/horacio/.conda/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
data.shape

(22870354, 8)

In [83]:
data.head()

,CONTENT_CATEGORY,CONTENT_CATEGORY_BOTTOM,CONTENT_CATEGORY_TOP,FEC_EVENT,ON_SITE_SEARCH_TERM,PAGE,SITE_ID,USER_ID
0,1,1,1,2018-03-30 07:35:48,1,1,1,0
1,2,2,2,2018-03-30 07:35:52,1,2,2,0
2,2,2,2,2018-03-30 07:36:11,1,3,3,0
3,2,2,2,2018-03-30 07:36:16,1,4,3,0
4,2,2,2,2018-03-30 07:41:38,1,5,2,0


In [84]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

MemoryError: 

In [ ]:
X_test.head(5)

In [ ]:
X_test.shape

In [ ]:
X_test.iloc[0,0:65].sum() # la suma de las frecuencias debe ser 1

In [ ]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

In [ ]:
X_train.sample(5)

In [ ]:
X_train.iloc[0,0:65].sum()

In [ ]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [ ]:
y_prev = pd.read_csv("../data/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(set(X_train.index))
y_train.loc[list(idx)] = 1

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(chi2,k=250)
fit = test.fit(X_train, y_train)
X_train = pd.DataFrame(fit.transform(X_train))
X_test = pd.DataFrame(fit.transform(X_test))

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

fi = []
test_probs = []
i = 0
for train_idx, valid_idx in model_selection.KFold(n_splits=10, shuffle=True).split(X_train):
    i += 1
    Xt = X_train.iloc[train_idx]
    yt = y_train.loc[X_train.index].iloc[train_idx]

    Xv = X_train.iloc[valid_idx]
    yv = y_train.loc[X_train.index].iloc[valid_idx]

    learner = LGBMClassifier(n_estimators=10000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)])
    
    test_probs.append(pd.Series(learner.predict_proba(X_test)[:, -1],
                                index=X_test.index, name="fold_" + str(i)))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
test_probs.index.name="USER_ID"
test_probs.name="SCORE"
test_probs.to_csv("respuesta", header=True, compression="zip")
fi = pd.concat(fi, axis=1).mean(axis=1)

### Experimental models

En este caso conv contiene los resultados para enviar al concurso, pero a continuacion subdividiremos X_train para poder usar el periodo de octubre a diciembre como testeo.

In [3]:
data = data[data.FEC_EVENT.dt.month < 10]
X_test2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test2 = pd.concat(X_test2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [4]:
data = data[(data.FEC_EVENT.dt.month < 10) & (data.FEC_EVENT.dt.month > 6)]
X_train2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train2 = pd.concat(X_train2, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [5]:
y_prev2 = pd.read_csv("../data/conversiones.csv")
y_test2 = pd.Series(0, index=X_test2.index)
idx = set(y_prev2[y_prev2.mes >= 10].USER_ID.unique()).intersection(set(X_test2.index))
y_test2.loc[list(idx)] = 1

In [6]:
y_train2 = pd.Series(0, index=X_train2.index)
idx = set(y_prev2[(y_prev2.mes < 10) & (y_prev2.mes > 6)].USER_ID.unique()).intersection(set(X_train2.index))
y_train2.loc[list(idx)] = 1

In [7]:
print(X_train2.shape, y_train2.shape, X_test2.shape, y_test2.shape)

(11356, 1434) (11356,) (11529, 2026) (11529,)


In [8]:
features = list(set(X_train2.columns).intersection(set(X_test2.columns)))
X_train2 = X_train2[features]
X_test2 = X_test2[features]

In [9]:
X_train2.shape

(11356, 1434)

### Decision Tree Classifier

In [10]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train2, y_train2)
parciales = dtc.predict(X_test2)

In [11]:
from sklearn.metrics import roc_auc_score

scores = roc_auc_score(y_test2,parciales)

In [12]:
scores

0.6586946689250622

In [86]:
# some feature selection

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(chi2,k=200)
fit = test.fit(X_train2, y_train2)
X_train2 = pd.DataFrame(fit.transform(X_train2))
X_test2 = pd.DataFrame(fit.transform(X_test2))

In [14]:
print(X_train2.shape, X_test2.shape)

(11356, 200) (11529, 200)


In [15]:
from sklearn.tree import DecisionTreeClassifier

dtc2 = DecisionTreeClassifier()
dtc2.fit(X_train2, y_train2)
parciales = dtc2.predict(X_test2)
scores = roc_auc_score(y_test2,parciales)

scores

0.6636116405828137

In [67]:
# Optimization through GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

pg = [
    {"splitter":["best","random"],
     "max_depth":[1,2,3,4,5,7,10], 
     "max_features":[10,50,100,150,200], 
     "max_leaf_nodes":[10,50,100,150,200]}
]

opt = GridSearchCV(dtc2, pg, scoring="roc_auc",n_jobs=-1,cv=StratifiedKFold(5))
opt.fit(X_train2, y_train2)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'splitter': ['best', 'random'], 'max_depth': [1, 2, 3, 4, 5, 7, 10], 'max_features': [10, 50, 100, 150, 200], 'max_leaf_nodes': [10, 50, 100, 150, 200]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [68]:
results = pd.DataFrame(opt.cv_results_)
results

/home/horacio/.conda/envs/ds/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/horacio/.conda/envs/ds/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/horacio/.conda/envs/ds/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/horacio

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_max_leaf_nodes,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.046188,0.006628,0.009473,0.001654,1,10,10,best,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.627162,0.706341,0.788467,0.610585,0.615124,0.669540,0.068889,317,0.671061,0.757821,0.825561,0.563819,0.598841,0.683421,0.097314
1,0.049966,0.009806,0.009636,0.001446,1,10,10,random,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.514970,0.517767,0.513987,0.646831,0.487408,0.536193,0.056395,346,0.529897,0.528098,0.513939,0.687476,0.505395,0.552961,0.067867
2,0.049100,0.003871,0.006907,0.001086,1,10,50,best,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.967341,0.980404,0.892011,0.981351,0.831193,0.930476,0.059620,225,0.974541,0.974374,0.921135,0.990967,0.806652,0.933534,0.067677
3,0.040320,0.004957,0.008811,0.001184,1,10,50,random,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.543848,0.537513,0.608238,0.564693,0.525714,0.556001,0.029019,343,0.524991,0.527513,0.581095,0.559451,0.527214,0.544053,0.022493
4,0.049522,0.010188,0.009767,0.001370,1,10,100,best,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.510676,0.946804,0.876653,0.521286,0.777004,0.726480,0.180194,300,0.531694,0.955283,0.927103,0.547704,0.723882,0.737133,0.179968
5,0.048707,0.010338,0.009373,0.000489,1,10,100,random,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.521573,0.512658,0.500000,0.561250,0.544468,0.527987,0.022078,348,0.559569,0.503195,0.503185,0.594673,0.542986,0.540721,0.034895
6,0.052986,0.015042,0.009299,0.000451,1,10,150,best,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.790083,0.511518,0.779920,0.945994,0.517406,0.708991,0.169392,306,0.857170,0.525746,0.815466,0.932170,0.517845,0.729679,0.173827
7,0.045399,0.006989,0.009210,0.000455,1,10,150,random,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.500000,0.506156,0.716484,0.601857,0.534712,0.571833,0.080842,341,0.503195,0.507109,0.688347,0.555876,0.528876,0.556681,0.068451
8,0.066079,0.026495,0.013387,0.005833,1,10,200,best,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.982969,0.819094,0.983190,0.594944,0.834266,0.842903,0.142425,273,0.993166,0.861958,0.980628,0.626306,0.801842,0.852780,0.134181
9,0.056248,0.013447,0.011777,0.007923,1,10,200,random,"{'max_depth': 1, 'max_features': 10, 'max_leaf...",0.500456,0.568645,0.505016,0.501368,0.507987,0.516698,0.026117,350,0.503308,0.554825,0.507900,0.503127,0.510477,0.515927,0.019648


In [69]:
opt.best_score_

0.9984777203172032

In [70]:
optimised_decision_tree = opt.best_estimator_
optimised_decision_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=100, max_leaf_nodes=150,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [71]:
optimised_decision_tree.predict(X_test2)

array([0, 0, 0, ..., 0, 0, 0])

In [78]:
sol = optimised_decision_tree.predict(X_test2)

In [79]:
sol

array([0, 0, 0, ..., 0, 0, 0])

In [189]:
scores = roc_auc_score(y_test2,sol)
scores

0.6591435019591736

### CATBOOSTING